# __Chapter 0. Preparing data__

- In this chapter, we are going to download the '__adjusted closing price__' of stocks listed in the stock index __Russel 1000__. 
- The reason that we use Russel 1000 instead of S&P 500 is that the number of stocks left is not many when we filter stocks with missing values from 1999 to 2019, if we use S&P 500.
- We are aware of that filtering stocks with missing values will cause __Survivorship bias__. This will be fixed in the future research
- The adjusted closing price is going to turn into daily returns of stocks and then be used when we optimize a portfolio.

#### __Description of Data__
- __Timespan :__ Nov.1999 - Nov.2019 (recent 20 years)
- __Stock index used :__ Russel 1000
- __Source :__ Yahoo Finance
- __Library used for downloading data__ : Pandas datareader

#### __Contents__

- [__Step 01. Loading stock data__](#Step-01.-Loading-stock-data)
- [__Step 02. Filtering tickers__](#Step-02.-Filtering-tickers)
- [__Step 03. Creating a daily_price_df__](#Step-03.-Creating-a-daily_price_df)
- [__Step 04. Grouping daily_return by a year__](#Step-04.-Grouping-daily_return-by-a-year)
- [__Step 05. Downloading industry information__](#Step-05.-Downloading-industry-information)

---

## Step 01. Loading stock data

__1. Importing required libraries__

In [1]:
# for importing libraries again that is already imported, just in case that any change is made in libraries
%load_ext autoreload
%autoreload 2

# libraries for general work
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# for downloading stock data
import pandas_datareader.data as pdr
import datetime

# for downloading industry information of each company
# source : https://github.com/davidastephens/pandas-finance
from pandas_finance import Equity

# Disable printing all warnings on python 
import warnings
warnings.filterwarnings("ignore")

__2. Loading Russel 1000 index constituents__
- loading the tickers consisting the Russel 1000 index from the beginning of __Nov.1999__ till the end of __Nov.2019__

In [2]:
russel1000_tickers = pd.read_csv("./russel1000_constituents.csv", header=None)
russel1000_tickers = list(russel1000_tickers.iloc[:,0])
russel1000_tickers[:5]

['TWOU', 'MMM', 'ABT', 'ABBV', 'ABMD']

In [3]:
len(russel1000_tickers)

968

__3. Setting dates of `start` and `end` (the start and end dates when we download stocks)__

In [4]:
# We will look at stock prices over the past year, starting at January 1, 2016
start = (1999, 11, 1)
start = datetime.datetime(*start)
end = (2019, 12, 1)
end = datetime.datetime(*end)

__4. Download stock data__

In [5]:
stock_data_dict = {}
missing_tickers = []

for ticker in tqdm(russel1000_tickers):
    try:
        stock_data = pdr.DataReader(ticker, 'yahoo', start, end)
        stock_data_dict[ticker] = stock_data
    except:
        print(f"The following ticker made an error : {ticker}")
        missing_tickers.append(ticker)

The following ticker made an error : AGN
The following ticker made an error : APY
The following ticker made an error : WTR
The following ticker made an error : BHGE
The following ticker made an error : BBT
The following ticker made an error : BRK.B
The following ticker made an error : BPR
The following ticker made an error : BFA
The following ticker made an error : BFB
The following ticker made an error : CBS
The following ticker made an error : CELG
The following ticker made an error : CTL
The following ticker made an error : CHK
The following ticker made an error : CXO
The following ticker made an error : CY
The following ticker made an error : DPS
The following ticker made an error : DNB
The following ticker made an error : DNKN
The following ticker made an error : ETFC
The following ticker made an error : EQC
The following ticker made an error : XOG
The following ticker made an error : FDC
The following ticker made an error : GLIBA
The following ticker made an error : GWR
The follo

In [6]:
len(stock_data_dict)

898

In [7]:
len(missing_tickers)

70

__5. Checking `missing_tickers` once again__

In [8]:
replaced_missing_tickers = []

for ticker in missing_tickers:
    replaced_ticker = ticker.replace('.','-')
    replaced_missing_tickers.append(replaced_ticker)

In [9]:
for ticker in tqdm(replaced_missing_tickers):
    try:
        stock_data = pdr.DataReader(ticker, 'yahoo', start, end)
        stock_data_dict[ticker] = stock_data
    except:
        print(f"The following ticker made an error : {ticker}")
        missing_tickers.append(ticker)

The following ticker made an error : AGN
The following ticker made an error : APY
The following ticker made an error : WTR
The following ticker made an error : BHGE
The following ticker made an error : BBT
The following ticker made an error : BPR
The following ticker made an error : BFA
The following ticker made an error : BFB
The following ticker made an error : CBS
The following ticker made an error : CELG
The following ticker made an error : CTL
The following ticker made an error : CHK
The following ticker made an error : CXO
The following ticker made an error : CY
The following ticker made an error : DPS
The following ticker made an error : DNB
The following ticker made an error : DNKN
The following ticker made an error : ETFC
The following ticker made an error : XOG
The following ticker made an error : FDC
The following ticker made an error : GLIBA
The following ticker made an error : GWR
The following ticker made an error : HCP
The following ticker made an error : HDS
The followi

In [10]:
len(stock_data_dict)

908

In [11]:
# Saving
with open('russel1000_raw_data_dict.pickle', 'wb') as f: # 
    pickle.dump(stock_data_dict, f, pickle.HIGHEST_PROTOCOL)

---

## Step 02. Filtering tickers

__1. Filtering tickers with missing value__

In [12]:
ticker_list_in_stock_data_dict = list(stock_data_dict.keys())
stock_data_dict_len_list = []

for ticker in ticker_list_in_stock_data_dict:
    stock_data_dict_len_list.append(len(stock_data_dict[ticker]))

In [13]:
maximum_len = pd.Series(stock_data_dict_len_list).max()
maximum_len

5053

In [14]:
filtered_stock_data_dict = {}

for ticker in ticker_list_in_stock_data_dict:
    if len(stock_data_dict[ticker]) == maximum_len:
        filtered_stock_data_dict[ticker] = stock_data_dict[ticker]

In [15]:
len(filtered_stock_data_dict)

571

In [19]:
# # Saving
# with open('russel1000_stock_data_dict.pickle', 'wb') as f: # 
#     pickle.dump(filtered_stock_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [20]:
# Loading
with open('russel1000_stock_data_dict.pickle', 'rb') as f:
    filtered_stock_data_dict = pickle.load(f)

---

## Step 03. Creating a daily_price_df

__1. Creating a dataframe of price : `daily_price_df`__

In [21]:
daily_price_dict = {}
filtered_tickers = list(filtered_stock_data_dict.keys())

for ticker in filtered_tickers:
    daily_price_dict[ticker] = filtered_stock_data_dict[ticker].loc[:,'Adj Close']

In [23]:
daily_price_df = pd.DataFrame.from_dict(daily_price_dict)
daily_price_df.index.name = None

daily_price_df.head(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,XLNX,YUM,BRK-B,EQC,HEI-A,LGF-A,NCR,X,VMI,JW-A
1999-11-01,27.326748,10.175731,10.2500,3.750,1.045129,17.013006,10.15625,21.740425,17.608480,8.143493,...,27.857941,5.373026,41.740002,7.317558,2.147241,1.423901,8.022765,19.023846,13.733675,11.674436
1999-11-02,27.581104,9.592007,10.3125,3.750,1.059064,16.888933,10.28125,22.126066,17.446932,8.382089,...,28.366455,5.290237,42.820000,7.418488,2.138686,1.385417,7.739959,19.689667,13.659440,11.630880
1999-11-03,27.472086,9.292256,10.2500,3.875,1.077644,17.075039,10.65625,21.571711,17.527702,8.496197,...,29.958338,5.199167,43.060001,7.317558,1.984700,1.539352,7.859035,19.499424,13.461479,11.238826


__2. Checking the dtypes of__ `daily_price_df`
- Need to check the datatypes of data included in dataframe because some of the numbers may not be numbers(floats, ints, etc) but actually strings.
- If some strings are mixed with numbers in the dataframe, some computations such as `.cov()` cannot be performed.

In [24]:
daily_price_df.dtypes.value_counts()

float64    571
dtype: int64

- The datatype `object` in pandas means string, while `float64` means float numbere.
- Source : <a href="https://pbpython.com/pandas_dtypes.html">_Overview of Pandas Data Types_</a>

In [26]:
# # Saving
# with open('russel1000_daily_price_df.pickle', 'wb') as f: # 
#     pickle.dump(daily_price_df, f, pickle.HIGHEST_PROTOCOL)

In [27]:
# Loading
with open('russel1000_daily_price_df.pickle', 'rb') as f:
    daily_price_df = pickle.load(f)

---

## Step 04. Grouping daily_return by a year

__1. Creating a dataframe of return : `daily_return_df`__

In [28]:
daily_return_df = daily_price_df.pct_change()
daily_return_df = daily_return_df.dropna(axis=0)
daily_return_df.head(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,XLNX,YUM,BRK-B,EQC,HEI-A,LGF-A,NCR,X,VMI,JW-A
1999-11-02,0.009308,-0.057364,0.006098,0.000000,0.013333,-0.007293,0.012308,0.017738,-0.009174,0.029299,...,0.018254,-0.015408,0.025874,0.013793,-0.003984,-0.027027,-0.035250,0.034999,-0.005405,-0.003731
1999-11-03,-0.003953,-0.031250,-0.006061,0.033333,0.017543,0.011019,0.036474,-0.025054,0.004629,0.013613,...,0.056118,-0.017215,0.005605,-0.013605,-0.072000,0.111111,0.015385,-0.009662,-0.014493,-0.033708
1999-11-04,0.002646,0.028863,0.021341,0.000000,0.008621,0.000908,-0.032258,-0.003352,0.006913,0.018315,...,0.012546,0.000000,0.004645,0.006896,0.000000,0.250000,0.024621,-0.002439,0.007352,0.019380


In [29]:
daily_return_df.tail(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,XLNX,YUM,BRK-B,EQC,HEI-A,LGF-A,NCR,X,VMI,JW-A
2019-11-26,-0.005933,0.011007,0.035667,-0.017078,0.008189,0.008582,-0.020106,-0.001597,-0.010399,0.007919,...,-0.006317,0.017753,-0.000730,0.008290,0.004159,0.025082,0.003598,-0.007977,0.005276,0.010036
2019-11-27,0.008155,0.000000,0.013024,0.011480,0.005723,0.003767,0.010772,0.007467,0.003503,0.001644,...,0.023126,0.008922,0.007309,0.005177,0.007374,0.018085,0.001494,0.018275,0.006438,0.009725
2019-11-29,-0.004924,0.000351,-0.003100,-0.013497,0.006424,0.001521,-0.006597,0.001059,-0.006748,0.000365,...,-0.006106,0.000298,-0.000907,-0.004847,0.007119,-0.022988,-0.020585,-0.058148,-0.004659,-0.010469


__2. Grouping price data by a year__ : `grouped_daily_price_df_list`

In [30]:
# past 252 days' daily returns are considered to calculate the covariance
seq_length = 252

# portfolio rebalnacing period is 60 days
rebalancing_period = 60
    
# sliced dataframes are stored in the list below temporarily
sliced_daily_return_df_list = []

# for validation
for i in range(0, (daily_return_df.shape[0]-(seq_length)+1), rebalancing_period):  # i gets bigger by 20 : 0, 20, 40,...
    sliced_daily_return_df = (daily_return_df.iloc[(i):(i+seq_length),:])
    sliced_daily_return_df_list.append(sliced_daily_return_df) # this is used for portfolio optimization, so can stay as pd.DataFrame

In [31]:
len(sliced_daily_return_df_list)

81

In [32]:
validation_daily_price_df_list = []
test_daily_price_df_list = []

# for validation
for i in range(len(sliced_daily_return_df_list)):
    if i <= (47-1):
        validation_daily_price_df_list.append(sliced_daily_return_df_list[i])
    elif i >= (47+4-1) : 
        test_daily_price_df_list.append(sliced_daily_return_df_list[i])

In [33]:
print(f"len(validation_daily_price_df_list) : {len(validation_daily_price_df_list)}")
print(f"len(test_daily_price_df_list) : {len(test_daily_price_df_list)}")

len(validation_daily_price_df_list) : 47
len(test_daily_price_df_list) : 31


In [35]:
# # Saving - validation data
# with open('./russel1000_daily_return_df_validation.pickle', 'wb') as f: 
#     pickle.dump(validation_daily_price_df_list, f, pickle.HIGHEST_PROTOCOL)

In [38]:
# # Saving - test data
# with open('./russel1000_daily_return_df_test.pickle', 'wb') as f: 
#     pickle.dump(test_daily_price_df_list, f, pickle.HIGHEST_PROTOCOL)

---

## Step 05. Downloading industry information

In [39]:
sector_dict = {}

sector_dict['Industrials'] = []
sector_dict['Healthcare'] = []
sector_dict['Communication Services'] = []
sector_dict['Technology'] = []
sector_dict['Utilities'] = []
sector_dict['Financial Services'] = []
sector_dict['Basic Materials'] = []
sector_dict['Real Estate'] = []
sector_dict['Consumer Defensive'] = []
sector_dict['Consumer Cyclical'] = []
sector_dict['Energy'] = []

In [40]:
for ticker in tqdm(filtered_tickers):
    try: 
        ticker_sector = Equity(ticker).sector
        sector_dict[ticker_sector].append(ticker)
    except:
        print(f"following ticker made an issue : {ticker}")

following ticker made an issue : EV
following ticker made an issue : GD
following ticker made an issue : VAR



In [41]:
sector_dict["Financial Services"].append('BBT')
sector_dict["Energy"].append('CHK')
sector_dict["Industrials"].append('GD')
sector_dict["Industrials"].append('JEC')
sector_dict["Financial Services"].append('STI')

In [42]:
industry_sum = 0

for industry in sector_dict.keys():
    print(f"{industry} : {len(sector_dict[industry])}")
    industry_sum+= int(len(sector_dict[industry]))

print()
print(f"Total number of tickers : {industry_sum}")

Industrials : 89
Healthcare : 65
Communication Services : 17
Technology : 67
Utilities : 30
Financial Services : 87
Basic Materials : 30
Real Estate : 48
Consumer Defensive : 37
Consumer Cyclical : 72
Energy : 31

Total number of tickers : 573


In [43]:
# Saving
with open('./russel1000_sector_dict.pickle', 'wb') as f: # 
    pickle.dump(sector_dict, f, pickle.HIGHEST_PROTOCOL)